<a href="https://colab.research.google.com/github/xxp-nlp/add_tag_algorithm/blob/main/2021_8_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import re

filename = '/content/CiWoTestdata-Correct.txt'

dependancy = re.compile(r'''(?:\*\s\d+\s) # キャプチャ対象外
             (-?\d+)      # 数字(係り先)
                      ''', re.VERBOSE)

class Morph:
    def __init__(self, line):
        cols = line.split(' ')
        self.surface = cols[0] # 表層形(surface)


class Chunk_correct:
    def __init__(self, morphs, dst, id):
        self.morphs = morphs
        self.dst  = dst  # 係り先文節インデックス番号
        self.id = id
        self.phrase = ''.join([morph.surface for morph in morphs]) # 文節
        self.trim_phrase = self.phrase.replace('、','')

sentences = []
chunks = []
morphs = []
ids = []
dic_correct = {}
i = 0
with open(filename, mode='r') as f:
  for line in f:  # 1行ずつ読込
    dependancies = dependancy.match(line)
    fields = line.split(' ')


    if fields[0] == '#':
      id = fields[1]
      continue

    # EOSまたは係り受け解析結果でない場合
    if not (line == 'EOS\n' or dependancies):
      morphs.append(Morph(line))


    # EOSまたは係り受け解析結果で、形態素解析結果がある場合
    elif len(morphs) > 0:
      chunks.append(Chunk_correct(morphs, dst, id))
      morphs = []  

    # 係り受け結果の場合
    if dependancies:
      dst = int(dependancies.group(1))

    # EOSで係り受け結果がある場合
    if line == 'EOS\n' and len(chunks) > 0:
      sentences.append(chunks)
      chunks = []
      dic_correct[i] = [sentences,id]
      i+=1

for i, chunk in enumerate(sentences[1]):
    print('{}: {}  係り先:{}'.format(i, chunk.trim_phrase, chunk.dst))

0: 首相は  係り先:12
1: 今春の  係り先:2
2: 統一地方選後に  係り先:5
3: 党全体での  係り先:4
4: 新党移行を  係り先:5
5: 目指す  係り先:6
6: 考えを  係り先:7
7: 強調  係り先:12
8: 離党など  係り先:10
9: 性急な  係り先:10
10: 行動への  係り先:11
11: 自重を  係り先:12
12: 求めた。  係り先:-1


In [11]:
filename = '/content/CiWoTestdata.txt'

dependancy = re.compile(r'''(?:\*\s\d+\s) # キャプチャ対象外
             (-?\d+)      # 数字(係り先)
                      ''', re.VERBOSE)

class Morph:
    def __init__(self, line):
        cols = line.split(' ')
        self.surface = cols[0] # 表層形(surface)


class Chunk_wrong:
    def __init__(self, morphs, dst, id):
        self.morphs = morphs
        self.dst  = dst  # 係り先文節インデックス番号
        self.id = id
        self.phrase = ''.join([morph.surface for morph in morphs]) # 文節
        self.trim_phrase = self.phrase.replace('、','')
        self.tag = 'KEEP'





dic_wrong = {}
sentences_wrong = []
chunks = []
morphs = []
ids = []
i = 0
with open(filename, mode='r') as f:
  for line in f:  # 1行ずつ読込
    dependancies = dependancy.match(line)
    fields = line.split(' ')


    if fields[0] == '#':
      id = fields[1]
      continue

    # EOSまたは係り受け解析結果でない場合
    if not (line == 'EOS\n' or dependancies):
      morphs.append(Morph(line))


    # EOSまたは係り受け解析結果で、形態素解析結果がある場合
    elif len(morphs) > 0:
      chunks.append(Chunk_wrong(morphs, dst, id))
      morphs = []  

    # 係り受け結果の場合
    if dependancies:
      dst = int(dependancies.group(1))

    # EOSで係り受け結果がある場合
    if line == 'EOS\n' and len(chunks) > 0:
      sentences_wrong.append(chunks)
      chunks = []
      dic_wrong[i] = [sentences_wrong,id]
      i+=1


for i, chunk in enumerate(sentences_wrong[1]):
    print('{}: {}   {}'.format(i, chunk.trim_phrase, chunk.tag))

0: 党全体での   KEEP
1: 新党移行を   KEEP
2: 今春の   KEEP
3: 統一地方選後に   KEEP
4: 目指す   KEEP
5: 考えを   KEEP
6: 強調   KEEP
7: 離党など   KEEP
8: 性急な   KEEP
9: 行動への   KEEP
10: 自重を   KEEP
11: 首相は   KEEP
12: 求めた。   KEEP


In [15]:
# scale the data
lenth = min(len(sentences),len(sentences_wrong))
correct = []
wrong = []

for i in range(len(sentences)):
  for j in range(len(sentences_wrong)):
    if dic_correct[i][1] == dic_wrong[j][1]:  
      correct.append(sentences[i])
      wrong.append(sentences_wrong[j])
      


In [16]:
# タグ付きアルゴリズム

dic2 = {}

for i in range(lenth):
#  print("sentence",i+1)

  for j, chunk in enumerate(wrong[i]):
    if wrong[i][j].trim_phrase not in dic2:
      dic2[wrong[i][j].trim_phrase] = [] 
    dic2[wrong[i][j].trim_phrase].append(j)
  

  for j, chunk in enumerate(correct[i]):
    if j == len(correct[i])-1:
      break

    for m in range(len(dic2[correct[i][j].trim_phrase])):
      for n in range(len(dic2[correct[i][j+1].trim_phrase])):

        if correct[i][j].dst == j+1 and dic2[correct[i][j].trim_phrase][m] == dic2[correct[i][j+1].trim_phrase][n] - 1: 
          wrong[i][dic2[correct[i][j].trim_phrase][m]].tag = "REDUCE"


        if dic2[correct[i][j].trim_phrase][m] < len(wrong[i])-1 and dic2[correct[i][j].trim_phrase][m] == dic2[correct[i][j+1].trim_phrase][n] + 1: 
          wrong[i][dic2[sentences[i][j].trim_phrase][m]].tag = "SWAP_B"
          wrong[i][dic2[sentences[i][j+1].trim_phrase][n]].tag = "SWAP_F"

  dic2 = {}

#    print(j,sentences[i][j].trim_phrase,sentences[i][j].dst)
#  print("------------------------")

In [97]:
# step1結果確認
for i in range(1,2):
  print("-------  sentence ", i+1, "-------")
  for j, chunk in enumerate(wrong[i]):
    print('{}: {}: {}'.format(j, wrong[i][j].trim_phrase, wrong[i][j].tag))

-------  sentence  2 -------
0: 党全体での: REDUCE
1: 新党移行を: KEEP
2: 今春の: REDUCE
3: 統一地方選後に: KEEP
4: 目指す: REDUCE
5: 考えを: REDUCE
6: 強調: KEEP
7: 離党など: KEEP
8: 性急な: REDUCE
9: 行動への: REDUCE
10: 自重を: KEEP
11: 首相は: KEEP
12: 求めた。: KEEP


In [18]:
# class Merged:
#   def __init__(self, trim_phrase, tag, dst):
#     self.trim_phrases = trim_phrase
#     self.tag = tag
#     self.dst = dst


#   def merge(self):
    


# trim_phrase = []
# tag = []
# dst = []


# for i in range(1):
#   merged_list = []
#   str1 = ''
#   for j, chunk in enumerate(wrong[i]):
#     str1 += wrong[i][j].trim_phrase
#     if (wrong[i][j].tag == 'KEEP'):
#       merged_list.append(Merged(str1,'KEEP',))
#       str1 = ''
      

#   #   trim_phrase.append(chunk.trim_phrase)
#   #   tag.append(chunk.tag)
#   #   dst.append(chunk.dst)
#   # Merged(trim_phrase,tag)
#   # trim_phrase = []
#   # tag = []
#   # dst = []

  

In [19]:
# class New_chunk:
#     def __init__(self, new_phrase, new_tag, new_dst):
#       self.new_phrase = new_phrase
#       self.new_tag = new_tag
#       self.new_dst = new_dst



# class Merged:
#   def __init__(self, j, phrase, trim_phrase, tag):
#     self.j = j
#     self.phrase = phrase
#     self.trim_phrases = trim_phrase
#     self.tag = tag

#   def merge(self):
#     new_phrase = []
#     new_tag = []
#     index = []
#     set1 = set([])
#     str1 = ''
#     list1 = []
#     for i in range(j+1):
        
#       if tag[i] == 'REDUCE':
#         set1.add(i)
#         set1.add(i+1)
#         list1 = []

#       if tag[i] == 'KEEP':
#         for i in set1:
#           str1 += phrase[i]
#         if len(str1)>0:
#           new_phrase.append(str1)
#           index.append(list(set1))
#           new_tag.append("KEEP")
#           str1 = ''
#           set1.clear()
#         else: 
#           new_phrase.append(phrase[i])
#           new_tag.append("KEEP")
#           index.append(i)

#     print(new_phrase,new_tag,index)


    
# # main

# phrase = []
# trim_phrase = []
# tag = []


# for i in range(1,2):
#   for j, chunk in enumerate(wrong[i]):
#     phrase.append(chunk.phrase)
#     trim_phrase.append(chunk.trim_phrase)
#     tag.append(chunk.tag)

#   Merged(j, phrase, trim_phrase, tag).merge()
#   phrase = []
#   trim_phrase = []
#   tag = []

  

In [63]:
class New_chunk:
    def __init__(self, new_chunk):
      self.new_phrase = new_chunk[0]
      self.new_tag = new_chunk[1]
      self.index = new_chunk[2]



class Merged:
  def __init__(self, j, phrase, trim_phrase, tag):
    self.j = j
    self.phrase = phrase
    self.trim_phrases = trim_phrase
    self.tag = tag

  def merge(self):
    new_phrase = []
    new_tag = []
    index = []
    set1 = set([])
    str1 = ''
    list1 = []
    for i in range(j+1):
        
      if tag[i] == 'REDUCE':
        set1.add(i)
        set1.add(i+1)
        list1 = []

      if tag[i] == 'KEEP':
        for i in set1:
          str1 += phrase[i]
        if len(str1)>0:
          new_phrase.append(str1)
          index.append(list(set1))
          new_tag.append("KEEP")
          str1 = ''
          set1.clear()
        else: 
          new_phrase.append(phrase[i])
          new_tag.append("KEEP")
          index.append(i)

      if tag[i] == 'SWAP_F':
        new_phrase.append(phrase[i+1])
        new_tag.append("KEEP")
        index.append(i+1)
      if tag[i] == 'SWAP_B':
        new_phrase.append(phrase[i-1])
        new_tag.append("KEEP")
        index.append(i-1)

    return new_phrase,new_tag,index


    
# main

phrase = []
trim_phrase = []
tag = []
new_chunks = []

for i in range(10):
  # 
  for j, chunk in enumerate(wrong[i]):
    phrase.append(chunk.phrase)
    trim_phrase.append(chunk.trim_phrase)
    tag.append(chunk.tag)

  new_chunk = Merged(j, phrase, trim_phrase, tag).merge()
  phrase = []
  trim_phrase = []
  tag = []

  new_chunks.append(New_chunk(new_chunk))


In [92]:
for i in range(1):
  print('{}: {}: {}: {}'.format(i, new_chunks[i].new_phrase, new_chunks[i].new_tag, new_chunks[i].index))

0: ['同社では、', '準役員クラス以上に', '能力重視型の年俸制を', 'すでに', '導入している。']: ['KEEP', 'KEEP', 'KEEP', 'KEEP', 'KEEP']: [0, 1, [2, 3], 4, 5]


In [110]:

index_list = []
for i in range(2):
  temp_list = []
  for j in range(len(new_chunks[i].index)):
    if isinstance(new_chunks[i].index[j],int):
      temp_list.append(new_chunks[i].index[j])
    else:
      temp_list.append(new_chunks[i].index[j][-1])
  index_list.append(temp_list)
  temp_list = []

for i in range(len(index_list)):
  print(index_list[i])

[0, 1, 3, 4, 5]
[1, 3, 6, 7, 10, 11, 12]


In [111]:
# step1結果確認
for i in range(1,2):
  
  for j in range(len(index_list[1])):
    index = index_list[i][j]
    print(index, wrong[1][index].trim_phrase)


1 新党移行を
3 統一地方選後に
6 強調
7 離党など
10 自重を
11 首相は
12 求めた。
